In [17]:
# all imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("imports")

imports


In [ ]:
# Load the dataset
data = np.load('threebody_dataset.npz')
X = data['X']
y = data['y']
labels = data['labels']

# Flatten time series data for classification
n_conditions, n_timesteps, n_features = X.shape
X_flat = X.reshape(n_conditions, n_timesteps * n_features)

# Normalise features with zero-std guard
mean = X_flat.mean(axis=0, keepdims=True)
std = X_flat.std(axis=0, keepdims=True)
# Prevent division by zero for constant features
std[std == 0] = 1
X_scaled = (X_flat - mean) / std

print("Normalized shape:", X_scaled.shape)
print("NaN count after scaling:", np.isnan(X_scaled).sum())

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/threebody_dataset.npz'

In [ ]:
# Prepare data: split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print("Train size:", X_train.shape, "Test size:", X_test.shape)

Train size: (240, 30966) Test size: (61, 30966)


In [ ]:
# create classifier

clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

print("Model trained")

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Predict and evaluate

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)

print(classification_report(y_test, y_pred, target_names=labels))

In [ ]:
import matplotlib.pyplot as plt

# Plot confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 6))
plt.title('Confusion Matrix')
im = plt.imshow(cm, interpolation='nearest')
plt.colorbar(im)
plt.xticks(range(len(labels)), labels, rotation=45)
plt.yticks(range(len(labels)), labels)

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha='center', va='center')

plt.tight_layout()
plt.show()